Install required packages

In [46]:
!pip install scikit-learn pandas joblib


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Imports

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

## Data processing

In [4]:
# Read in the CSVs
real = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')

In [4]:
# Quick look
real.head(6)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"


In [14]:
# Assign categorisation
real['label'] = 1
fake['label'] = 0

In [15]:
# Concat datasets
data = pd.concat([real, fake], axis=0).reset_index(drop=True)
data = data[['title', 'label']]

In [7]:
data.tail(5)

,title,text,subject,date,real
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",False
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",False
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",False
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",False
44897,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",False


In [16]:
# Shuffle
data = data.sample(frac=1)
data

,title,label
22216,BREAKING: GOP Chairman Grassley Has Had Enoug...,0
27917,Failed GOP Candidates Remembered In Hilarious...,0
25007,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,0
1377,California AG pledges to defend birth control ...,1
32476,AZ RANCHERS Living On US-Mexico Border Destroy...,0
...,...,...
11284,Nigeria says U.S. agrees delayed $593 million ...,1
44732,Boiler Room #62 – Fatal Illusions,0
38158,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,0
860,Republican tax plan would deal financial hit t...,1


## Logistic Regression

In [48]:
X = data['title'].to_list()
y = data['label'].to_list()

In [49]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
# Convert text to numerical representation using TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))  # Unigrams & bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [51]:
# Train a Logistic Regression model
model = LogisticRegression(max_iter=500)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=500)

In [52]:
# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.9536
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.96      4669
           1       0.95      0.96      0.95      4311

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980



In [53]:
# Save the model and vectorizer for later use
joblib.dump(model, "news_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [54]:
# Function to make predictions on new headlines
def predict_news(headline):
    tfidf_input = vectorizer.transform([headline])
    prediction = model.predict(tfidf_input)[0]
    return "Real news" if prediction == 1 else "Fake news"

In [59]:
# Example usage
print(predict_news("Hillary Clinton suggests jailing Americans for posting 'misinformation'"))
print(predict_news("Pfizer CEO says third Covid vaccine dose likely needed within 12 months"))
print(predict_news("Starmer says US 'backstop' needed for Ukraine peace deal"))

Fake News
Real News
Real News
